In [5]:
import os
import uuid
import cv2
import mediapipe as mp


### Set the label you want to collect(make it same as folder name)

In [6]:
label = "rock"


### Path to save file

In [7]:
save_dir = os.path.join(os.getcwd(), label)
os.makedirs(save_dir, exist_ok=True)

print(f"Images will be saved to: {save_dir}")

Images will be saved to: c:\Users\Vihas\Documents\Pytesting\AI_RPS\rock


In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)

cap = cv2.VideoCapture(0)
image_count = 0

print(f"Collecting images for: {label}")
print("Press 'c' to capture, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    hand_crop = None
    if result.multi_hand_landmarks:
        h, w, _ = frame.shape
        for hand_landmarks in result.multi_hand_landmarks:
            x_coords = [lm.x for lm in hand_landmarks.landmark]
            y_coords = [lm.y for lm in hand_landmarks.landmark]
            x_min = int(min(x_coords) * w) - 20
            x_max = int(max(x_coords) * w) + 20
            y_min = int(min(y_coords) * h) - 20
            y_max = int(max(y_coords) * h) + 20

            x_min = max(x_min, 0)
            y_min = max(y_min, 0)
            x_max = min(x_max, w)
            y_max = min(y_max, h)

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            hand_crop = frame[y_min:y_max, x_min:x_max]

    cv2.putText(frame, f"Saved: {image_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("RPS Hand Capture", frame)
    key = cv2.waitKey(1)

    if key == ord('c') and hand_crop is not None:
        filename = f"{label}_{uuid.uuid4()}.jpg"
        filepath = os.path.join(save_dir, filename)
        cv2.imwrite(filepath, hand_crop)
        image_count += 1
        print(f"Saved: {filepath} (Total: {image_count})")

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()


Press 'c' to capture, 'q' to quit.
